## Extract Images from the tar file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf /content/drive/MyDrive/Project/ip102_v1.1.tar

## Read the files to store images based on the class

In [ ]:
import os
import pandas as pd

# list to store the names of the image files
train_file_names = []
test_file_name = []
val_file_name = []

# List tostore the actual lables of each image
train_actual_labels = []
test_actual_labels = []
val_actual_labels = []

# Read train, test and val file to get the list of files names for each categories

train_file = open("ip102_v1.1/train.txt")
for l in train_file:
    train_file_names.append(l.split(" ")[0])
    train_actual_labels.append(int(l.split(" ")[-1][:-1]))
train_file.close()

test_file = open("ip102_v1.1/test.txt")
for l in test_file:
    test_file_name.append(l.split(" ")[0])
    test_actual_labels.append(int(l.split(" ")[-1][:-1]))
test_file.close()

val_file = open("ip102_v1.1/val.txt")
for l in val_file:
    val_file_name.append(l.split(" ")[0])
    val_actual_labels.append(int(l.split(" ")[-1][:-1]))
val_file.close()

In [ ]:
# Create the folder to store the images
os.mkdir("train")
os.mkdir("test")
os.mkdir("val")

In [ ]:
# Creating list to convert the label to actual name of categories
super_calss = ["Rice",
               "Corn",
               "Wheat",
               "Beet",
               "Alfalfa",
               "Vitis",
               "Citrus",
               "Mango"
              ]
super_class_count = [
    14,13,9,8,13,16,19,10
]

In [ ]:
# This will continue previous process
class_cetegories = []
prev = 0
for i in range(0,len(super_class_count)):
    cl = list(range(prev,prev+super_class_count[i]))
    class_cetegories.append(cl)
    prev = prev + super_class_count[i]

This code is used to group classes based on their similarity or shared characteristics, such as belonging to the same type of object or activity. It is commonly used in image classification tasks to group similar classes together and improve the model's accuracy.





In [ ]:
# Create the subclass in the train, test and val folders to store images
for c in super_calss:
    os.mkdir("train/"+c)
    os.mkdir("test/"+c)
    os.mkdir("val/"+c)
    
class_cetegories

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
 [27, 28, 29, 30, 31, 32, 33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43],
 [44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56],
 [57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72],
 [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91],
 [92, 93, 94, 95, 96, 97, 98, 99, 100, 101]]

In [ ]:
# Define function to map the label number ot the image name
def find_category(inp_cat):
    for i in range(len(class_cetegories)):
        if inp_cat in class_cetegories[i]:
            return super_calss[i]
        

In [ ]:
# Move images from the image folder to the train, test, and val folder
import shutil
root_image_folder = "ip102_v1.1/images/"

# Following for loop to move images to the trianing folder
for i in range(len(train_file_names)):
    name = train_file_names[i]
    label = train_actual_labels[i]
    subfolder = find_category(label)  
    shutil.move(root_image_folder+name,"train/"+subfolder+"/"+name)
    # shutil.move("train/"+subfolder+"/"+name,"val/"+subfolder+"/"+name)

# Following for loop is to move image to the test folder
for i in range(len(test_file_name)):
    name = test_file_name[i]
    label = test_actual_labels[i]
    subfolder = find_category(label)
    shutil.move(root_image_folder+name,"test/"+subfolder+"/"+name)
    # shutil.move("test/"+subfolder+"/"+name,"val/"+subfolder+"/"+name)

# Folloing for loop is to move images to the validation folder
for i in range(len(val_file_name)):
    name = val_file_name[i]
    label = val_actual_labels[i]
    subfolder = find_category(label)
    shutil.move(root_image_folder+name,"val/"+subfolder+"/"+name)
    # shutil.move("val/"+subfolder+"/"+name,"val/"+subfolder+"/"+name)

In [ ]:
def add_noise(inputs):
     noise = torch.randn_like(inputs)*0.2
     return inputs + noise

## Import libraries

In [ ]:
!pip install barbar
import torch
import os
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.optim as optim
import argparse
from distutils.util import strtobool
from barbar import Bar
import copy
import time


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# ResNet50 Pretrained Model

In [ ]:
# Load Pretrained ResNet50 Model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)


dropout = 0.5
num_ftrs = model.fc.in_features

# Add new layer to make it to classify for 8 super class
model.fc = nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(num_ftrs, 8))

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# Define the Image Data

In [ ]:
# Following is the image loader
input_size = 224
# Following is the data augmentation Which will be applied during the training
data_transforms = {
            'train': transforms.Compose([
               transforms.Resize(256),
               transforms.AugMix(),
                transforms.RandomCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }


# Following is the dataset loader to save the memoery
batch_size = 64  # Can  be adjusted based on the available ram

train_folder = "train/" # Training Folder
val_folder = "val/"  # Validation Data Folder

train_set = ImageFolder(root= train_folder, transform= data_transforms['train'])
valid_set = ImageFolder(root= val_folder, transform= data_transforms['val'])

# Following is data generator defining the batch size
train_set = DataLoader(train_set, batch_size= batch_size, shuffle= True, 
                            num_workers= 8, pin_memory=True)
valid_set = DataLoader(valid_set, batch_size= batch_size, shuffle= False, 
                                num_workers= 8, pin_memory= True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Prepare the model for the trianing

In [ ]:
# Following is the model training procedure
val_acc_history = []
train_acc_history = []
val_loss_history = []
train_loss_history = []


device = "cuda"
model = model.to(device)
weight_decay =  0.00001
dataloaders = {'train': train_set, 'val': valid_set}
criterion = nn.CrossEntropyLoss()
params_to_update = model.parameters()
optimizer_ft = optim.Adam(params_to_update, lr= 0.0001, betas= (0.9, 0.999),
                                eps= 1e-08, weight_decay= weight_decay)



# Training The model using the for loop method

In [ ]:
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma= 0.96)
optimizer = optimizer_ft
num_epochs = 5
is_save_checkpoint = False
is_inception = False
ckpepoch = 0
since = time.time()
best_acc = 0.0

# Following are the epochs loop of the training 
for epoch in range(ckpepoch, num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        running_loss = 0.0
        running_corrects = 0

        for data in Bar(dataloaders[phase]):
            if len(data) > 2:
                inputs, _ , labels = data
                inputs = add_noise(inputs)
                inputs = inputs.to(device)
            else:
                inputs, labels = data
                inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                if is_inception and phase == 'train':
                    outputs, aux_outputs = model(inputs)
                    loss1 = criterion(outputs, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4*loss2
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            if is_save_checkpoint:
                torch.save(model.state_dict(), "/content/drive/MyDrive/Project/"+"resnet50pretrainedSuperModel_100")
                # torch.save({'epoch': epoch,
                #         'model_state_dict': model.state_dict(),
                #         'optimizer_state_dict': optimizer.state_dict(),
                #         'best_acc': best_acc,
                #         'scheduler_state_dict': scheduler.state_dict(),
                #         }, checkpointFn)

            count2stop = 0

        elif phase == 'val':
            count2stop += 1

        if phase == 'val':
            val_loss_history.append(epoch_loss)
            val_acc_history.append(epoch_acc)
        if phase == 'train':
            train_loss_history.append(epoch_loss)
            train_acc_history.append(epoch_acc)


    if count2stop == 10:
        break

    if scheduler:
        scheduler.step()
        print('lr :', scheduler.get_lr())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

model.load_state_dict(best_model_wts)
his = {'train_loss': train_loss_history,
       'train_acc': train_acc_history,
       'val_loss': val_loss_history,
       'val_acc': val_acc_history}

Epoch 0/4
----------
45095/45095: [===============================>] - ETA 12.5s
train Loss: 0.9350 Acc: 0.6791
7508/7508: [===============================>] - ETA 5.0s
val Loss: 0.8076 Acc: 0.7207
lr : [9.216e-05]

Epoch 1/4
----------


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:587: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


45095/45095: [===============================>] - ETA 13.7s
train Loss: 0.7862 Acc: 0.7311
7508/7508: [===============================>] - ETA 6.4s
val Loss: 0.8001 Acc: 0.7344
lr : [8.847359999999999e-05]

Epoch 2/4
----------
45095/45095: [===============================>] - ETA 12.0s
train Loss: 0.6709 Acc: 0.7698
7508/7508: [===============================>] - ETA 4.6s
val Loss: 0.7557 Acc: 0.7525
lr : [8.493465599999999e-05]

Epoch 3/4
----------
45095/45095: [===============================>] - ETA 13.5s
train Loss: 0.5932 Acc: 0.8000
7508/7508: [===============================>] - ETA 5.3s
val Loss: 0.7366 Acc: 0.7572
lr : [8.153726975999998e-05]

Epoch 4/4
----------
45095/45095: [===============================>] - ETA 14.7s
train Loss: 0.5241 Acc: 0.8223
7508/7508: [===============================>] - ETA 5.1s
val Loss: 0.7377 Acc: 0.7686
lr : [7.827577896959998e-05]

Training complete in 101m 36s
Best val Acc: 0.768647


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Project/"+"resnet50pretrainedSuperModel_8_with_noise")